In [1]:
from datetime import date
import numpy as np
import tensorflow as tf
from tensorflow import keras

2024-01-29 09:57:10.752236: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 09:57:11.139897: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-29 09:57:11.139939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-29 09:57:11.219862: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-29 09:57:11.390126: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 09:57:11.392909: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
MONTHS_DICT = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
              }

In [3]:
def generate_data(n_samples):
    X, y = [], []
    ordinal_min = date(1000, 1, 1).toordinal()
    ordinal_max = date(9999, 12, 31).toordinal()
    
    ordinal_random = np.random.randint(ordinal_max - ordinal_min, size=n_samples) + ordinal_min

    for ordinal in ordinal_random:
        dt = date.fromordinal(ordinal)
        month = MONTHS_DICT[dt.month]
        y.append(dt.isoformat())
        X.append(month + " " + dt.strftime("%d, %Y"))
        
    return X,y

In [4]:
INPUT_CHARS = sorted(set("".join(MONTHS_DICT.values()) + "1234567890, "))
OUTPUT_CHARS = "0123456789-"
def vectorize_input(data):
    return [INPUT_CHARS.index(char) for char in data]

def vectorize_output(data):
    return [OUTPUT_CHARS.index(char) for char in data]

In [5]:
def create_dataset(n_samples, batch_size=32):
    x, y = generate_data(n_samples)
    
    X = [vectorize_input(dt) for dt in x]
    Y = [vectorize_output(dt) for dt in y]
    X, Y = tf.ragged.constant(X, ragged_rank=1), tf.ragged.constant(Y,ragged_rank=1)

    X, Y = (X + 1).to_tensor(), (Y + 1).to_tensor()
    
    dataset = tf.data.Dataset.from_tensor_slices((X,Y))
    # dataset = dataset.shuffle(n_samples)
    dataset = dataset.batch(batch_size)
    
    return dataset, X.shape, Y.shape
    


In [6]:
train_dataset, MAX_INPUT_SHAPE, MAX_OUTPUT_SHAPE = create_dataset(n_samples=15000)
test_dataset, _, _ = create_dataset(n_samples=3000)
val_dataset, _, _ = create_dataset(n_samples=2000)

2024-01-29 09:57:20.709400: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-29 09:57:20.904870: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
# DELETE
for x,y in train_dataset.take(1):
    print(x)

tf.Tensor(
[[17 21 33 23 26  1  5  3  2  1  5  8  6  7  0  0  0  0]
 [16 36 30 24  1  5  4  2  1  8 11  5  5  0  0  0  0  0]
 [13 32 33 27 28  1  3  4  2  1  4 10  4 11  0  0  0  0]
 [17 21 33 23 26  1  4  3  2  1 11  6 11  4  0  0  0  0]
 [16 36 28 38  1  5  9  2  1  5 10  5  4  0  0  0  0  0]
 [19 23 35 31 22 24 33  1  5 12  2  1  6  8  3  9  0  0]
 [16 21 30 36 21 33 38  1  3 12  2  1  8  6 10 12  0  0]
 [13 32 33 27 28  1  3  6  2  1  7  8  5  9  0  0  0  0]
 [13 32 33 27 28  1  5 12  2  1 10  5  9 10  0  0  0  0]
 [17 21 33 23 26  1  3  5  2  1  9  4  4  7  0  0  0  0]
 [14 24 23 24 29 22 24 33  1  3  8  2  1 10  8  6  3  0]
 [15 24 22 33 36 21 33 38  1  3 11  2  1 10  7  6  5  0]
 [17 21 33 23 26  1  5 12  2  1  4  7 12  9  0  0  0  0]
 [20 24 32 35 24 29 22 24 33  1  3 10  2  1  5  4  3  4]
 [19 23 35 31 22 24 33  1  4  6  2  1 11 12  9  8  0  0]
 [17 21 33 23 26  1  6  4  2  1  7  9  5 11  0  0  0  0]
 [16 21 30 36 21 33 38  1  5  8  2  1 11 12 10  6  0  0]
 [20 24 32 35 24 29 

In [8]:
embedding_size = 32

encoder = keras.models.Sequential([
    keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1, output_dim=embedding_size, input_shape=[None]),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")   
])

model = keras.models.Sequential([
    encoder,
    keras.layers.RepeatVector(MAX_OUTPUT_SHAPE[1]),
    decoder
])

In [9]:
optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
469/469 [==============================] - 16s 28ms/step - loss: 1.5509 - accuracy: 0.4436 - val_loss: 1.1259 - val_accuracy: 0.5977
Epoch 2/10
469/469 [==============================] - 12s 27ms/step - loss: 0.9017 - accuracy: 0.6721 - val_loss: 0.6950 - val_accuracy: 0.7362
Epoch 3/10
469/469 [==============================] - 12s 26ms/step - loss: 0.5879 - accuracy: 0.7654 - val_loss: 0.4670 - val_accuracy: 0.8110
Epoch 4/10
469/469 [==============================] - 13s 27ms/step - loss: 0.4844 - accuracy: 0.8160 - val_loss: 0.3264 - val_accuracy: 0.8686
Epoch 5/10
469/469 [==============================] - 12s 26ms/step - loss: 0.2467 - accuracy: 0.9093 - val_loss: 0.1924 - val_accuracy: 0.9356
Epoch 6/10
469/469 [==============================] - 12s 26ms/step - loss: 0.1248 - accuracy: 0.9646 - val_loss: 0.0883 - val_accuracy: 0.9789
Epoch 7/10
469/469 [==============================] - 12s 26ms/step - loss: 0.0747 - accuracy: 0.9842 - val_loss: 0.0407 - val_accuracy:

In [10]:
def prepare_input(date_strs):
    ids = [vectorize_input(str) for str in date_strs]
    X = tf.ragged.constant(ids, ragged_rank=1)
    return (X + 1).to_tensor()

In [11]:
X_new = prepare_input(["September 17, 2009", "July 14, 1789"])

In [12]:
ids = np.argmax(model.predict(X_new), axis=-1)

1/1 [==============================] - 0s 452ms/step


In [13]:
for id in ids:
    print(''.join([OUTPUT_CHARS[index -1] for index in id]))

2009-09-17
1789-07-14


In [14]:
X_new = prepare_input(["May 02, 2020", "July 14, 1789"])

In [15]:
ids = np.argmax(model.predict(X_new), axis=-1)
for id in ids:
    print(''.join([OUTPUT_CHARS[index -1] for index in id]))

1/1 [==============================] - 0s 444ms/step
2000-02-02
1789-09-14


In [16]:
X_new = prepare_input(["May 02, 2020", "September 17, 2009"])

In [17]:
X_new[0]

<tf.Tensor: shape=(18,), dtype=int32, numpy=
array([17, 21, 38,  1,  3,  5,  2,  1,  5,  3,  5,  3,  0,  0,  0,  0,  0,
        0], dtype=int32)>

In [18]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [vectorize_input(dt) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor()

In [19]:
def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < MAX_INPUT_SHAPE[1]:
        X = tf.pad(X, [[0, 0], [0, MAX_INPUT_SHAPE[1] - X.shape[1]]])
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    ids = np.argmax(model.predict(X), axis=-1)
    return ids

In [20]:
pred = convert_date_strs(["July 14, 1789", "May 01, 2020", "August 01, 1993", "November 14, 1996", 
                           "May 22, 4322", "May 01, 1999", "July 14, 4111"])

1/1 [==============================] - 0s 19ms/step


In [21]:
for id in pred:
    print(''.join([OUTPUT_CHARS[index -1] for index in id]))

1789-07-14
2020-05-01
1993-08-01
1996-11-14
4322-05-22
1999-05-01
4111-07-14


## Teacher Forcing

In [22]:
SOS_TOKEN = len(OUTPUT_CHARS) + 1
EOS_TOKEN = SOS_TOKEN + 1
def shift_output(dataset):
    X = np.concatenate([X for X,Y in dataset], axis=0)
    Y = np.concatenate([Y for X,Y in dataset], axis=0)

    sos_token = tf.fill(dims=(Y.shape[0], 1), value=SOS_TOKEN)
    X_decoder = np.concatenate([sos_token, Y[:, :-1]], axis=1)
    
    return tf.constant(X), X_decoder, tf.constant(Y)

In [23]:
X_train, X_train_decoder, y_train = shift_output(train_dataset)
X_test, X_test_decoder, y_test = shift_output(test_dataset)
X_val, X_val_decoder, y_val = shift_output(val_dataset)

In [24]:
encoder_input_layer = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding_layer = keras.layers.Embedding(input_dim=len(INPUT_CHARS)+ 1, output_dim=512)(encoder_input_layer)
output, encoder_h_state, encoder_c_state = keras.layers.LSTM(128, return_state=True)(encoder_embedding_layer)

encoder_state = [encoder_h_state, encoder_c_state]

decoder_input_layer = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding_layer = keras.layers.Embedding(input_dim=len(OUTPUT_CHARS)+ 2, output_dim=512)(decoder_input_layer)
decoder_LSTM_output = keras.layers.LSTM(128, return_sequences=True)(decoder_embedding_layer, initial_state=encoder_state)
decoder_output = keras.layers.Dense(len(OUTPUT_CHARS)+1, activation="softmax")(decoder_LSTM_output)

model = keras.models.Model(inputs=[encoder_input_layer, decoder_input_layer], outputs=[decoder_output])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit([X_train, X_train_decoder], y_train, epochs=20, validation_data=([X_val, X_val_decoder], y_val))

Epoch 1/20
469/469 [==============================] - 21s 39ms/step - loss: 0.6839 - accuracy: 0.7673 - val_loss: 0.1474 - val_accuracy: 0.9667
Epoch 2/20
469/469 [==============================] - 18s 38ms/step - loss: 0.0805 - accuracy: 0.9836 - val_loss: 0.0314 - val_accuracy: 0.9973
Epoch 3/20
469/469 [==============================] - 18s 38ms/step - loss: 0.0250 - accuracy: 0.9974 - val_loss: 0.0468 - val_accuracy: 0.9888
Epoch 4/20
469/469 [==============================] - 18s 38ms/step - loss: 0.0436 - accuracy: 0.9911 - val_loss: 0.0137 - val_accuracy: 0.9995
Epoch 5/20
469/469 [==============================] - 18s 38ms/step - loss: 0.0069 - accuracy: 0.9999 - val_loss: 0.0042 - val_accuracy: 1.0000
Epoch 6/20
469/469 [==============================] - 18s 38ms/step - loss: 0.0255 - accuracy: 0.9948 - val_loss: 0.0119 - val_accuracy: 0.9991
Epoch 7/20
469/469 [==============================] - 18s 38ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.0029 - val_accuracy:

In [25]:
test_dates = ["July 14, 1789", "May 01, 2020"]

In [26]:
def predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = tf.fill(dims=[len(X), 1], value=SOS_TOKEN)
    for index in range(MAX_OUTPUT_SHAPE[1]):
        X_decoder = tf.pad(Y_pred, [[0,0],[0, MAX_OUTPUT_SHAPE[1] - Y_pred.shape[1]]])
        pred_indices = np.argmax(model.predict([X, X_decoder])[:,index:index+1], axis=-1)
        Y_pred = np.concatenate((Y_pred, pred_indices), axis=1)
    return Y_pred

In [27]:
y_pred = predict_date_strs(["July 14, 1789", "May 01, 2020", "August 01, 1993", "November 14, 1996", 
                           "May 22, 4322", "May 01, 1999", "July 14, 4111"])

1/1 [==============================] - 0s 20ms/step


In [28]:
for dt in y_pred[:,1:]:
    print(''.join([OUTPUT_CHARS[index -1] for index in dt]))

1789-07-14
2020-05-01
1993-08-01
1996-11-14
4322-05-22
1999-05-01
4111-07-14


### TFA seq2seq

In [29]:
import tensorflow_addons as tfa

/home/nick/miniconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/nick/miniconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file

#### Training

In [30]:
EMBEDDING_SIZE = 512
encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding_layer = keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1, output_dim=EMBEDDING_SIZE)
encoder_embedding = encoder_embedding_layer(encoder_input)

encoder_output, encoder_hidden_state, encoder_cell_state = keras.layers.LSTM(128, return_state=True)(encoder_embedding)
encoder_state = [encoder_hidden_state, encoder_cell_state]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding_layer = keras.layers.Embedding(input_dim=len(OUTPUT_CHARS) + 2, output_dim=EMBEDDING_SIZE)
decoder_embedding = decoder_embedding_layer(decoder_input)

sampler = tfa.seq2seq.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(128)
output_layer = keras.layers.Dense(len(OUTPUT_CHARS)+1)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
    cell=decoder_cell,
    sampler=sampler,
    output_layer=output_layer
)

final_output, final_state, final_lengths = decoder(decoder_embedding, initial_state=encoder_state)
Y_proba = keras.layers.Activation("softmax")(final_output.rnn_output)

model = keras.models.Model(inputs=[encoder_input, decoder_input], outputs=[Y_proba])
optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit([X_train, X_train_decoder], y_train, epochs=20, validation_data=([X_val, X_val_decoder], y_val))

Epoch 1/20
469/469 [==============================] - 21s 39ms/step - loss: 0.7107 - accuracy: 0.7545 - val_loss: 0.1536 - val_accuracy: 0.9625
Epoch 2/20
469/469 [==============================] - 18s 38ms/step - loss: 0.0720 - accuracy: 0.9870 - val_loss: 0.0422 - val_accuracy: 0.9941
Epoch 3/20
469/469 [==============================] - 17s 37ms/step - loss: 0.0206 - accuracy: 0.9985 - val_loss: 0.0099 - val_accuracy: 0.9998
Epoch 4/20
469/469 [==============================] - 17s 36ms/step - loss: 0.0396 - accuracy: 0.9927 - val_loss: 0.0090 - val_accuracy: 0.9997
Epoch 5/20
469/469 [==============================] - 17s 36ms/step - loss: 0.0055 - accuracy: 0.9999 - val_loss: 0.0044 - val_accuracy: 0.9999
Epoch 6/20
469/469 [==============================] - 17s 36ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 7/20
469/469 [==============================] - 17s 36ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy:

In [31]:
y_pred = predict_date_strs(["July 14, 1789", "May 01, 2020"])

1/1 [==============================] - 0s 18ms/step


In [32]:
for dt in y_pred[:,1:]:
    print(''.join([OUTPUT_CHARS[index -1] for index in dt]))

1789-07-14
2020-05-01


### GreedyEmbedding

#### Inference

In [33]:
inference_sampler = tfa.seq2seq.GreedyEmbeddingSampler(embedding_fn=decoder_embedding_layer)
inference_decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
    cell=decoder_cell,
    sampler=inference_sampler,
    output_layer=output_layer,
    maximum_iterations= MAX_OUTPUT_SHAPE[1]
)
batch_size = tf.shape(encoder_input)[:1]
start_tokens = tf.fill(dims=batch_size, value=SOS_TOKEN)
final_outputs, final_state, final_sequence_lengths = inference_decoder(
    start_tokens,
    initial_state=encoder_state,
    start_tokens=start_tokens,
    end_token=0)

In [34]:
inference_model =keras.models.Model(inputs=[encoder_input], outputs=[final_outputs.sample_id])

In [35]:
def fast_predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = inference_model.predict(X)
    return Y_pred

In [36]:
y_pred = fast_predict_date_strs(["July 14, 1789", "May 01, 2020"])

1/1 [==============================] - 0s 367ms/step
